In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import numpy as np
import jsonlines
from termcolor import colored
from sklearn.metrics import classification_report

In [ ]:
LABELS = ['B-MUS', 'B-PER', 'I-MUS', 'I-PER', 'O']    

original format:
```
{"id": 50, "sent_items": [["A", "O"], ["large", "O"], ["ensemble", "O"], ["of", "O"], ["trombonists", "O"], ["would", "O"], ["gather", "O"], ["to", "O"], ["play", "O"], ["music", "O"], ["written", "O"], ["for", "O"], ["multiple", "O"], ["trombones", "O"], ["or", "O"], ["transcribed", "O"], ["from", "O"], ["other", "O"], ["sources", "O"], [",", "O"], ["such", "O"], ["as", "O"], ["the", "O"], ["chorales", "O"], ["of", "O"], ["Johann", "B-MUS"], ["Sebastian", "I-MUS"], ["Bach", "I-MUS"], [".", "O"]]}
```

In [ ]:
def convert_dev_from_json_to_conll(dev_path, conll_path):
    with jsonlines.open(dev_path, "r") as fin:
        with open(conll_path, "w") as fout:
            for line in fin:
                tokens = line["sent_items"]
                print(tokens)
                print([f"{t[0]} {t[1]}\n" for t in tokens])
                for t in tokens:
                    fout.write(f"{t[0]} {t[1]}\n")
                    
                    
def get_set_labels(set_path):
    set_labels = []
    with open(set_path, "r") as f:
        for t in f.readlines():
            if t.strip():
                label = t.split()[-1]
                set_labels.append(label)
            else:
                print(t)
    return set_labels
                    


In [ ]:
dev_path = '../data/musicians_dataset/dev_converted.jsonl'
conll_path = '../experiments/gold_devs/dev_converted.conll'


convert_dev_from_json_to_conll(dev_path, conll_path)



In [ ]:
gold_path = '../experiments/gold_devs/dev_converted.conll'

pred_path = '../experiments/predictions/sents_with_no_ents_bigger-manual.conll'
gold_labels = get_set_labels(gold_path)
pred_labels = get_set_labels(pred_path)
print(set(gold_labels))
print(set(pred_labels))


confusion_matrix(gold_labels, pred_labels, labels=['B-MUS', 'B-PER', 'I-MUS', 'I-PER', 'O'])

In [ ]:
dev_path = '../data/musicians_dataset/split_dev_20211118.jsonl'
conll_path = '../experiments/gold_devs/split_dev_20211118.conll'


convert_dev_from_json_to_conll(dev_path, conll_path)

In [ ]:
len(['The O\n', 'Polish O\n', 'NREAP O\n', 'plan O\n', 'is O\n', 'targeting O\n', '6,700 O\n', 'MW O\n', 'of O\n', 'wind O\n', 'power O\n', 'by O\n', '2020 O\n', 'whilst O\n', 'EWEA O\n', "'s O\n", '2009 O\n', 'forecast O\n', 'suggests O\n', 'a O\n', 'higher O\n', 'wind O\n', 'capacity O\n', 'of O\n', '10–12 O\n', '\xa0 O\n', 'GW O\n', 'is O\n', 'possible O\n', '. O\n'])

In [ ]:
gold_path = '../experiments/gold_devs/split_dev_20211118.conll'

pred_path = '../experiments/predictions/sents_with_no_ents_bigger-split.conll'
gold_labels = get_set_labels(gold_path)
pred_labels = get_set_labels(pred_path)
print(set(gold_labels) - set(pred_labels))
print()


confusion_matrix(gold_labels, pred_labels, labels=LABELS)

In [ ]:
import re

def untag_dev_sentences(devpath):
    clean_sentences = []
    tokens = []
    with open(devpath, "r") as fin:
        for line in fin.readlines():
            sentence = re.sub('\-\[[A-Z]*\]',  '', line)
            clean_sentences.append(sentence)
            for token in sentence.split():
                if token.strip():
                    tokens.append(token)
    return clean_sentences, tokens

In [ ]:
clean_sentences, tokens = untag_dev_sentences("../data/musicians_dataset/split_dev_2.jsonl")

In [ ]:
tokens
# len(gold_labels)

### manual

         ['B-MUS', 'B-PER', 'I-MUS', 'I-PER', 'O'] 
['B-MUS', [  29,   61,    0,    1,    7],
 'B-PER',[   1,   158,    0,    6,   14],
 'I-MUS'[    1,    0,    32,   51,    4],
 'I-PER',[   0,    0,    0,   106,    8],
 'O']    [   1,   29,    1,   15,   4021]
 

### Slit
 
 

In [ ]:
from collections import Counter

In [ ]:
dev_musicians = []
with jsonlines.open('../data/musicians_dataset/split_dev_only_hearst.jsonl') as f:
    for sentence in f:
        musician = ""
        for item in sentence["sent_items"]:
            if "-MUS" in item[1]:
                musician += item[0] + " "
            elif item[1] == "O" and musician != "":
                dev_musicians.append(musician[:-1])
                musician = ""

In [ ]:
train_musicians = []
with jsonlines.open('../data/musicians_dataset/split_train_only_hearst_uniques.jsonl') as f:
    for sentence in f:
        musician = ""
        for item in sentence["sent_items"]:
            if "-MUS" in item[1]:
                musician += item[0] + " "
            elif item[1] == "O" and musician != "":
                train_musicians.append(musician[:-1])
                musician = ""

In [ ]:
mandev_musicians = []
with jsonlines.open('../data/musicians_dataset/dev_converted.jsonl') as f:
    for sentence in f:
        musician = ""
        for item in sentence["sent_items"]:
            if "-MUS" in item[1]:
                musician += item[0] + " "
            elif item[1] == "O" and musician != "":
                mandev_musicians.append(musician[:-1])
                musician = ""

In [ ]:
print(len(mandev_musicians), len(dev_musicians))

In [ ]:
dev_musicians

In [ ]:
Counter(train_musicians).most_common(100)

In [ ]:
repetitions_in_dev = 0
repetitions_in_train = 0

mandev_musicians_set = set(mandev_musicians)
for x in Counter(train_musicians).most_common(100):
    if x[0] in mandev_musicians_set:
        print(x)
        repetitions_in_train += x[1]
        repetitions_in_dev += mandev_musicians.count(x[0])

print(repetitions_in_dev, repetitions_in_train)

In [ ]:
print(len(dev_musicians), len(mandev_musicians))

In [ ]:
repetitions_in_dev = 0
repetitions_in_train = 0

dev_musicians_set = set(dev_musicians)
for x in Counter(train_musicians).most_common(100):
    if x[0] in dev_musicians_set:
        print(x)
        repetitions_in_train += x[1]
        repetitions_in_dev += dev_musicians.count(x[0])

print(repetitions_in_dev, repetitions_in_train)

In [ ]:
len(dev_musicians)

In [ ]:
dev_musicians_set = set(dev_musicians)
for x in Counter(train_musicians).most_common(100):
    if x[0] in dev_musicians_set:
        print(x)

In [ ]:
PREDICTIONS_PATH = "../experiments/predictions"

In [ ]:
with open(f"{PREDICTIONS_PATH}/only_hearst_uniques-manual.conll", 'r') as f:
    sentence = []
    for line in f.readlines():
        word, tag = line.strip().split(" ", 1)
        if word == ".":
            color_text = [x[0] if x[1] == "O" else colored(x[0], 'red') if "PER" in x[1] else colored(x[0], 'green') for x in sentence]
            print(" ".join(color_text))
            print()
            sentence = []
        else:
            sentence.append([word, tag])


In [ ]:
with jsonlines.open(f"../data/musicians_dataset/split_dev_only_hearst_uniques.jsonl", 'r') as f:
    for i, line in enumerate(f):
        color_text = [x[0] if x[1] == "O" else colored(x[0], 'red') if "PER" in x[1] else colored(x[0], 'green') for x in line["sent_items"]]
        print(i, line["id"], " ".join(color_text))
        print()

In [ ]:
bad_df = pd.read_csv("../tests/fixtures/bad_model_gold_and_predictions.tsv", sep="\t", names=["gold", "pred"])

df["match"] = "_"

In [ ]:
df.to_csv("../experiments/predictions/hit_or_miss.csv", sep=" ")

In [ ]:
golds = df["gold"].tolist()
preds = df["pred"].tolist()

classification_report(golds, preds, labels=["B-PER", "B-MUS", "I-PER", "I-MUS", "O"])
            
        

In [ ]:
def tag_hits(df, pos, neg):
    neg_labels = ["O", f"B-{neg}", f"I-{neg}"]
    for i, row in df.iterrows():
        if row["match"] == "_":
            if (row["gold"] in neg_labels) and (row["pred"] in neg_labels):
                df.at[i,'match'] = "tn"
            elif (row["gold"] in neg_labels) and (row["pred"].endswith(pos)):
                df.at[i,'match'] = "fp"
            elif (row["gold"].endswith(pos)) and not (row["pred"].endswith(pos)):
                df.at[i,'match'] = "fn"
            elif (row["gold"] == f"B-{pos}") and (row["pred"] == f"B-{pos}"):
                span = []
                x = i+1
                error = False
                while df.at[x, "gold"] == f"I-{pos}":
                    span.append(x)
                    x += 1
                    if df.at[x, "gold"] != df.at[x, "pred"]:
                        error = True                    
                if error:
                    df.at[i,'match'] = "fn"
                else:
                    df.at[i,'match'] = "tp"
                for x in span:
                    df.at[x,'match'] = "--"


def get_span_recall(filepath, positive_label, negative_label):
    labels = [f"B-{positive_label}", f"I-{positive_label}", f"B-{negative_label}", f"I-{negative_label}", "O"]
    df = pd.read_csv(filepath, sep="\t", names=["gold", "pred"])
    golds = df["gold"].tolist()
    preds = df["pred"].tolist()
    report  = classification_report(golds, preds)
    print(report)
    df["match"] = "_"
    tag_hits(df, positive_label, negative_label)
    print("true positive spans:", df[df["match"] == "tp"].count())
    print("total gold positives:", df[df["gold"] == f"B-{positive_label}"].count())

In [ ]:
get_span_recall("../tests/fixtures/bad_model_gold_and_predictions.tsv", "MUS", "PER")

In [ ]:
get_span_recall("../tests/fixtures/good_model_gold_and_predictions.tsv", "MUS", "PER")

In [ ]:
df[df["match"] == "tp"].count()

In [ ]:
df[df["gold"] == "B-MUS"].count()

In [ ]:
34/115*100

In [ ]:
for i, row in df.iterrows():
    if row["match"] == "_":
        if (row["gold"] in ["O", "B-PER", "I-PER"]) and (row["pred"] in ["O", "B-PER", "I-PER"]):
            df.at[i,'match'] = "tn"
        elif (row["gold"] in ["O", "B-PER", "I-PER"]) and (row["pred"].endswith("MUS")):
            df.at[i,'match'] = "fp"
        elif (row["gold"].endswith("MUS")) and not (row["pred"].endswith("MUS")):
            df.at[i,'match'] = "fn"
        elif (row["gold"] == "B-MUS") and (row["pred"] == "B-MUS"):
            span = []
            x = i+1
            error = False
            while df.at[x, "gold"] == "I-MUS":
                span.append(x)
                x += 1
                if df.at[x, "gold"] != df.at[x, "pred"]:
                    print(df.at[x, :])
                    error = True                    
            if error:
                df.at[i,'match'] = "fn"
            else:
                df.at[i,'match'] = "tp"
            for x in span:
                df.at[x,'match'] = "--"
        
        

In [ ]:
results = []
for x in range(len(golds)):
    if golds[x] == "O" and preds[x] == "O":
        results.append(("O", "tp"))
    elif golds[x] == "O" and preds[x] != "O":
        results.append(("O", "fp"))
        

'        precision    recall  f1-score   support\n\n
B-PER       0.80      0.86      0.83       613\n  
B-MUS       0.77      0.30      0.43       115\n       
I-PER       0.78      0.91      0.84       537\n       
I-MUS       0.81      0.32      0.46       134\n
O           0.99      0.99      0.99      7982\n\n    

accuracy                        0.96      9381\n   
macro avg   0.83      0.68      0.71      9381\n
weighted avg 0.96      0.96      0.95      9381\n'

In [17]:
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import argparse
from simpletransformers.ner import NERModel
import jsonlines
import numpy as np
import json
from IPython.display import Markdown, display, HTML


In [3]:
model = NERModel(
    "roberta", "../experiments/bigger_hearst-with_expansion/best_model"
)

In [19]:
gold_tags = []
pred_tags = []

with jsonlines.open("../data/schools/split_dev_expand.jsonl", "r") as f:
    for idx, line in enumerate(f):
        gold_color_text = []
        for x in line["sent_items"]:
            if x[1] == "O":
                gold_color_text.append(x[0])
            elif "ORG" in x[1]:
                gold_color_text.append(Markdown(f"**{x[0]}**"))
            else:
                gold_color_text.append(Markdown(f"*{x[0]}*"))
                
#         gold_color_text = [x[0] if x[1] == "O" else Markdown(f"**{x[0]}**") if "ORG" in x[1] else colored(x[0], 'green') for x in line["sent_items"]]
        print("gold: ", display(" ".join(gold_color_text)))
        words = [w[0] for w in line["sent_items"]]
        tokens = [w[1] for w in line["sent_items"]]
#         print(" ".join(words))
        predictions, raw_outputs = model.predict([" ".join(words)])
        pred = predictions[0]
        pred_color_text = [list(x.keys())[0] if "O" in x.values() else colored(list(x.keys())[0] , 'red') if "ORG" in list(x.values())[0] else colored(list(x.keys())[0] , 'green') for x in pred]
        print("pred: ", " ".join(pred_color_text))
        print()

gold:  <IPython.core.display.HTML object>


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

pred:  Following years of negotiation , aimed at establishing a permanent international tribunal to prosecute individuals accused of genocide and other serious international crimes , such as crimes against humanity , war crimes and crimes of aggression , the United Nations General Assembly convened a five - week diplomatic conference in Rome in June 1998 " to finalize and adopt a convention on the establishment of an international criminal court " .

gold:  <IPython.core.display.HTML object>


  0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

pred:  Staffler studied biochemistry at the Technical University of Munich from 2001 until 2006 .

gold:  <IPython.core.display.HTML object>


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

pred:  Oster was still in contact with Halder and Witzleben , although Witzleben had been transferred to Frankfurt am Main , reducing his ability to lead a coup attempt .



In [20]:
gold_tags = []
pred_tags = []

with jsonlines.open("../data/schools/split_dev_expand.jsonl", "r") as f:
    for idx, line in enumerate(f):
        gold_color_text = [x[0] if x[1] == "O" else colored(x[0], 'red') if "ORG" in x[1] else colored(x[0], 'green') for x in line["sent_items"]]
        print("gold: ", " ".join(gold_color_text))
        words = [w[0] for w in line["sent_items"]]
        tokens = [w[1] for w in line["sent_items"]]
#         print(" ".join(words))
        predictions, raw_outputs = model.predict([" ".join(words)])
        pred = predictions[0]
        pred_color_text = [list(x.keys())[0] if "O" in x.values() else colored(list(x.keys())[0] , 'red') if "ORG" in list(x.values())[0] else colored(list(x.keys())[0] , 'green') for x in pred]
        print("pred: ", " ".join(pred_color_text))
        print()

gold:  Following years of negotiation , aimed at establishing a permanent international tribunal to prosecute individuals accused of genocide and other serious international crimes , such as crimes against humanity , war crimes and crimes of aggression , the United Nations General Assembly convened a five - week diplomatic conference in Rome in June 1998 " to finalize and adopt a convention on the establishment of an international criminal court " .


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

pred:  Following years of negotiation , aimed at establishing a permanent international tribunal to prosecute individuals accused of genocide and other serious international crimes , such as crimes against humanity , war crimes and crimes of aggression , the United Nations General Assembly convened a five - week diplomatic conference in Rome in June 1998 " to finalize and adopt a convention on the establishment of an international criminal court " .

gold:  He played seven seasons for the Chicago Bears and two for the Birmingham Stallions of the United States Football League ( USFL ) .


  0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

pred:  Staffler studied biochemistry at the Technical University of Munich from 2001 until 2006 .

gold:  Oster was still in contact with Halder and Witzleben , although Witzleben had been transferred to Frankfurt am Main , reducing his ability to lead a coup attempt .


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

pred:  Oster was still in contact with Halder and Witzleben , although Witzleben had been transferred to Frankfurt am Main , reducing his ability to lead a coup attempt .



In [21]:
! ls ../data/schools/

dataset_expand.jsonl	split_dev_school.jsonl	 split_train_expand.jsonl
dataset_school.jsonl	split_test_expand.jsonl  split_train_school.jsonl
split_dev_expand.jsonl	split_test_school.jsonl


In [23]:
train_captures = set()

with jsonlines.open("../data/schools/split_train_expand.jsonl", "r") as f:
    for line in f:
        capture = ""
        for item in line['sent_items']:
            if item[1] == "B-SCHOOL":
                capture = item[0]
            elif item[1] == "I-SCHOOL":
                capture += " " + item[0]
            elif capture != "":
                train_captures.add(capture)
                capture = ""
print(train_captures)

{'the Student Advisory Board', 'Sonoma State University', 'the College of Communication', 'the American University of Beirut', 'University of New South Wales', 'Blue Ribbon School / National School of Excellence', 'Ferrum College', 'the Higher Technological Institute', 'Wawasan Open University', 'El Circo de La Mega', 'the Los Angeles Cultural Heritage Commission', 'the Divinity School at the University of Oxford', 'Ross School of Business', 'Church of St. Joseph', 'Grossmont High School', 'Marist Brothers High School', 'North Carolina Central', 'the Royal College of Art', 'Oxford University Press Journal', 'Norwood District High School', 'the State Department', 'the Universidade Federal da Bahia', 'the Ohio State', 'New Hampshire Route', 'St Phillips Christian College', 'University of Pennsylvania School of Dental Medicine', 'The Sports Network', 'the " New York Times', 'Gujarat National Law University', 'University of Oxford', 'the French Culinary Institute', 'the Beaverbrook Art Gal

In [26]:
total = 0
valid = 0

with jsonlines.open("../data/schools/split_dev_expand.jsonl", "r") as f:
    for line in f:
        total += 1
        capture = ""
        found = False
        for item in line['sent_items']:
            if item[1] == "B-SCHOOL":
                capture = item[0]
            elif item[1] == "I-SCHOOL":
                capture += " " + item[0]
            elif capture != "":
                if capture in train_captures:
                    found = True
                capture = ""
        if not found:
            valid += 1
            
print(total, valid)

300 215
